In [1]:
# !pip install gensim scikit-learn pandas numpy torch tqdm

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from collections import Counter
import gensim

# --- UČITAVANJE I PODJELA PODATAKA ---
full_df = pd.read_csv('TRAIN.csv')

# Učitaj sve test skupove
test1_df = pd.read_csv('test-1.csv')
test2_df = pd.read_csv('test-2.csv')
test3_df = pd.read_csv('test-3.csv')

def get_text_column(df):
    for col in df.columns:
        if col.lower() in ['sentence', 'text']:
            return col
    raise ValueError("Nema stupca 'Sentence' ili 'Text'!")

text_col = get_text_column(full_df)

# Stratified split: 95% train, 5% valid
train_df, valid_df = train_test_split(full_df, test_size=0.05, stratify=full_df['Label'], random_state=42)

# --- TOKENIZACIJA I VOKABULAR ---
def tokenize(text):
    return text.lower().split()

counter = Counter()
for text in train_df[text_col]:
    counter.update(tokenize(text))
vocab = {word: idx+2 for idx, (word, _) in enumerate(counter.most_common())}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

# --- EMBEDDING ---
from gensim.models.fasttext import load_facebook_model

embedding_path = 'cc.hr.300.bin'
ft_model = load_facebook_model(embedding_path)
embeddings = ft_model.wv  

embedding_dim = embeddings.vector_size
embedding_matrix = np.zeros((len(vocab), embedding_dim))
for word, idx in vocab.items():
    if word in embeddings:
        embedding_matrix[idx] = embeddings[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim, ))

# --- DATASET ---
class TextDataset(Dataset):
    def __init__(self, df, text_col, vocab, max_len=50):
        self.texts = df[text_col].tolist()
        self.labels = df['Label'].tolist()
        self.vocab = vocab
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        tokens = tokenize(self.texts[idx])
        ids = [self.vocab.get(token, self.vocab['<unk>']) for token in tokens][:self.max_len]
        ids += [self.vocab['<pad>']] * (self.max_len - len(ids))
        return torch.tensor(ids), torch.tensor(self.labels[idx])

max_len = 50
batch_size = 32
train_ds = TextDataset(train_df, text_col, vocab, max_len)
valid_ds = TextDataset(valid_df, text_col, vocab, max_len)

test1_text_col = get_text_column(test1_df)
test2_text_col = get_text_column(test2_df)
test3_text_col = get_text_column(test3_df)

test1_ds = TextDataset(test1_df, test1_text_col, vocab, max_len)
test2_ds = TextDataset(test2_df, test2_text_col, vocab, max_len)
test3_ds = TextDataset(test3_df, test3_text_col, vocab, max_len)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)
test1_dl = DataLoader(test1_ds, batch_size=batch_size)
test2_dl = DataLoader(test2_ds, batch_size=batch_size)
test3_dl = DataLoader(test3_ds, batch_size=batch_size)

# --- MODELI ---
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=256, num_classes=3, dropout=0.8):
        super().__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        out = self.dropout(hidden[-1])
        return self.fc(out)

class GRUClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=256, num_classes=3, dropout=0.8):
        super().__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.gru(x)
        out = self.dropout(hidden[-1])
        return self.fc(out)

class CNNClassifier(nn.Module):
    def __init__(self, embedding_matrix, num_filters=128, kernel_sizes=[3,4,5], num_classes=3, dropout=0.8):
        super().__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embedding_dim)) for k in kernel_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)
    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]
        x = [torch.max(pool, dim=2)[0] for pool in x]
        x = torch.cat(x, dim=1)
        x = self.dropout(x)
        return self.fc(x)

# --- TRENING I VALIDACIJA ---
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def eval_model(model, dataloader, device, criterion=None, return_loss=False):
    model.eval()
    preds, targets = [], []
    total_loss = 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            if criterion and return_loss:
                loss = criterion(logits, y)
                total_loss += loss.item()
            pred = logits.argmax(1).cpu().numpy()
            preds.extend(pred)
            targets.extend(y.cpu().numpy())
    if return_loss and criterion:
        return np.array(preds), np.array(targets), total_loss / len(dataloader)
    return np.array(preds), np.array(targets)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def run_training(model_class, name, epochs=50, dropout=0.8, lr=5e-4):
    print(f"\n{name} training...")
    model = model_class(embedding_matrix, dropout=dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        train_loss = train_epoch(model, train_dl, optimizer, criterion, device)
        _, _, val_loss = eval_model(model, valid_dl, device, criterion, return_loss=True)
        if (epoch+1) % 5 == 0 or epoch == 0:
            print(f"{name} Epoch {epoch+1}: Train loss {train_loss:.4f} | Validation loss {val_loss:.4f}")
    results = {}
    for test_name, test_dl in zip(
        ['test1', 'test2', 'test3'],
        [test1_dl, test2_dl, test3_dl]
    ):
        preds, targets = eval_model(model, test_dl, device)
        report = classification_report(targets, preds, digits=4, output_dict=True, target_names=["positive", "neutral", "negative"])
        matrix = confusion_matrix(targets, preds)
        print(f"\n{name} on {test_name} Classification report:\n", classification_report(targets, preds, digits=4, target_names=["positive", "neutral", "negative"]))
        print(f"{name} on {test_name} Confusion matrix:\n", matrix)
        results[test_name] = {
            'precision': report['macro avg']['precision'],
            'recall': report['macro avg']['recall'],
            'f1': report['macro avg']['f1-score'],
            'accuracy': report['accuracy'],
            'confusion_matrix': matrix.tolist(),
            'full_report': classification_report(targets, preds, digits=4, target_names=["positive", "neutral", "negative"])
        }
    return results

# --- POKRETANJE ---
lstm_results = run_training(LSTMClassifier, "LSTM", epochs=50, dropout=0.8, lr=5e-4)
gru_results = run_training(GRUClassifier, "GRU", epochs=50, dropout=0.8, lr=5e-4)
cnn_results = run_training(CNNClassifier, "CNN", epochs=50, dropout=0.8, lr=5e-4)

# --- SPREMANJE ---
with open('results.md', 'w', encoding='utf-8') as f:
    for model_name, results in [('LSTM', lstm_results), ('GRU', gru_results), ('CNN', cnn_results)]:
        f.write(f"## {model_name}\n\n")
        for test_name, res in results.items():
            f.write(f"### {test_name}\n")
            f.write(f"- Precision: {res['precision']:.4f}\n")
            f.write(f"- Recall: {res['recall']:.4f}\n")
            f.write(f"- F1: {res['f1']:.4f}\n")
            f.write(f"- Accuracy: {res['accuracy']:.4f}\n")
            f.write(f"- Confusion matrix: {res['confusion_matrix']}\n\n")
            f.write(f"Full classification report:\n{res['full_report']}\n\n")


/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



LSTM training...
LSTM Epoch 1: Train loss 0.9225 | Validation loss 0.9003
LSTM Epoch 5: Train loss 0.9064 | Validation loss 0.8971
LSTM Epoch 10: Train loss 0.9024 | Validation loss 0.8979
LSTM Epoch 15: Train loss 0.9013 | Validation loss 0.8975
LSTM Epoch 20: Train loss 0.9458 | Validation loss 0.9297
LSTM Epoch 25: Train loss 0.9019 | Validation loss 0.9019
LSTM Epoch 30: Train loss 0.9034 | Validation loss 0.9004
LSTM Epoch 35: Train loss 0.9002 | Validation loss 0.9023
LSTM Epoch 40: Train loss 0.8989 | Validation loss 0.9054
LSTM Epoch 45: Train loss 0.8987 | Validation loss 0.9030
LSTM Epoch 50: Train loss 0.8978 | Validation loss 0.9007


/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()


LSTM on test1 Classification report:
               precision    recall  f1-score   support

    positive     0.0000    0.0000    0.0000       165
     neutral     0.6585    1.0000    0.7941       430
    negative     0.0000    0.0000    0.0000        58

    accuracy                         0.6585       653
   macro avg     0.2195    0.3333    0.2647       653
weighted avg     0.4336    0.6585    0.5229       653

LSTM on test1 Confusion matrix:
 [[  0 165   0]
 [  0 430   0]
 [  0  58   0]]


/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()


LSTM on test2 Classification report:
               precision    recall  f1-score   support

    positive     0.0000    0.0000    0.0000       216
     neutral     0.5816    1.0000    0.7355       431
    negative     0.0000    0.0000    0.0000        94

    accuracy                         0.5816       741
   macro avg     0.1939    0.3333    0.2452       741
weighted avg     0.3383    0.5816    0.4278       741

LSTM on test2 Confusion matrix:
 [[  0 216   0]
 [  0 431   0]
 [  0  94   0]]


/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ivancarevic/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()


LSTM on test3 Classification report:
               precision    recall  f1-score   support

    positive     0.0000    0.0000    0.0000       267
     neutral     0.3317    1.0000    0.4981       263
    negative     0.0000    0.0000    0.0000       263

    accuracy                         0.3317       793
   macro avg     0.1106    0.3333    0.1660       793
weighted avg     0.1100    0.3317    0.1652       793

LSTM on test3 Confusion matrix:
 [[  0 267   0]
 [  0 263   0]
 [  0 263   0]]

GRU training...
GRU Epoch 1: Train loss 0.9163 | Validation loss 0.8981
GRU Epoch 5: Train loss 0.9048 | Validation loss 0.8972
GRU Epoch 10: Train loss 0.8214 | Validation loss 0.8023
GRU Epoch 15: Train loss 0.7494 | Validation loss 0.7687
GRU Epoch 20: Train loss 0.6789 | Validation loss 0.7580
GRU Epoch 25: Train loss 0.5857 | Validation loss 0.8096
GRU Epoch 30: Train loss 0.4784 | Validation loss 0.9778
GRU Epoch 35: Train loss 0.3589 | Validation loss 1.1809
GRU Epoch 40: Train loss 0.261